[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/diogoflim/MGP/blob/main/GP/q1.ipynb)

# Um Problema de Mix de Produção

$$\max z = 0,3x_1 + 0,25x_2 + 0,2x_3$$
                                            sujeito a:
$$0,22x_1 + 0,52x_2 + 0,74x_3 \leq 9600000$$
$$0,52x_1 + 0,34x_2 + 0,20x_3 \leq 4800000$$
$$0,28x_1 + 0,14x_2 + 0,06x_3 \leq 2200000$$ 
$$x_3 \geq 16x_1$$
$$x_2 \leq 600000$$
$$x_i \geq 0,   \forall i \in \{1,2,3\}$$
  

In [ ]:
#Execute esse bloco caso esteja executando no Google Colab
!pip install -q pyomo
!apt-get install -y -qq glpk-utils

In [1]:
import pyomo.environ as pe

## Modelando diretamente

In [2]:
M=pe.ConcreteModel()
M.x = pe.Var([1,2,3], within=pe.NonNegativeReals)
M.z = pe.Objective(rule = 0.3*M.x[1] + 0.25*M.x[2] + 0.2*M.x[3], sense=pe.maximize)
M.R1 = pe.Constraint(rule = 0.22*M.x[1]+ 0.52 * M.x[2] + 0.74*M.x[3] <= 9600000)
M.R2 = pe.Constraint(rule = 0.5*M.x[1]+ 0.34 * M.x[2] + 0.2*M.x[3] <= 4800000)
M.R3 = pe.Constraint(rule = 0.28*M.x[1]+ 0.14 * M.x[2] + 0.06*M.x[3] <= 2200000)
M.R4 = pe.Constraint(rule = M.x[3] >= 16*M.x[1])
M.R5 = pe.Constraint(rule = M.x[2] <= 600000)
pe.SolverFactory('glpk').solve(M)
print(f"z= {pe.value(M.z)}")
for k in M.x.keys(): 
    print (f"x_{k} = {pe.value(M.x[k])}")

z= 2845522.3880597027
x_1 = 770149.253731343
x_2 = 600000.0
x_3 = 12322388.0597015


## Modelando a partir dos dados

As vezes pode ser importante inicializar os parâmetros no modelo. Assim, o modelo...

In [3]:
import pandas as pd

In [4]:
dados = pd.DataFrame({'pura':[0.22, 0.52, 0.74],
                      'octana':[0.5, 0.34, 0.2],
                      'aditivo': [0.28, 0.14, 0.06]},
                      index = ['verde', 'azul', 'comum'])
dados


,pura,octana,aditivo
verde,0.22,0.50,0.28
azul,0.52,0.34,0.14
comum,0.74,0.20,0.06


In [5]:
print(list(dados.index))
print(list(dados.columns))

['verde', 'azul', 'comum']
['pura', 'octana', 'aditivo']


In [7]:
comp_dict = dados.stack().to_dict()
comp_dict

{('verde', 'pura'): 0.22,
 ('verde', 'octana'): 0.5,
 ('verde', 'aditivo'): 0.28,
 ('azul', 'pura'): 0.52,
 ('azul', 'octana'): 0.34,
 ('azul', 'aditivo'): 0.14,
 ('comum', 'pura'): 0.74,
 ('comum', 'octana'): 0.2,
 ('comum', 'aditivo'): 0.06}

In [8]:
capacidades = {'pura':9600000, 'octana': 4800000, 'aditivo': 2200000}
lucros = {'verde':0.3, 'azul': 0.25, 'comum': 0.2}

In [9]:
M = pe.ConcreteModel()

In [10]:
# Conjuntos
M.tipos_prod = pe.Set(initialize = list(dados.index)) #tipos de produto que podem ser produzidos
M.mp = pe.Set(initialize = list(dados.columns)) #matérias-primas para a produção


In [11]:
#Parâmetros
M.comp = pe.Param(M.tipos_prod, M.mp, initialize=comp_dict) #composição dos produtos
M.capacidades = pe.Param(M.mp, initialize= capacidades) #capacidades
M.lucros = pe.Param(M.tipos_prod, initialize= lucros) #lucros
M.capacidade_azul = pe.Param(initialize = 600000)


In [12]:
M.x = pe.Var(M.tipos_prod, within = pe.NonNegativeReals)

In [13]:
def Lucro(M):
    return sum(M.x[i] * M.lucros[i] for i in M.tipos_prod)

M.z = pe.Objective(rule = Lucro, sense = pe.maximize)

In [14]:
M.R_capacidades = pe.ConstraintList()

for j in M.mp:
    M.R_capacidades.add (expr = sum(M.x[i] * M.comp[i, j] for i in M.tipos_prod) <= M.capacidades[j])

M.R2 = pe.Constraint(expr = M.x['comum'] >= 16 * M.x['verde'])

M.R3 = pe.Constraint(expr = M.x['azul'] <= M.capacidade_azul)

In [15]:
pe.SolverFactory('glpk').solve(M)

{'Problem': [{'Name': 'unknown', 'Lower bound': 2845522.3880597, 'Upper bound': 2845522.3880597, 'Number of objectives': 1, 'Number of constraints': 6, 'Number of variables': 4, 'Number of nonzeros': 13, 'Sense': 'maximize'}], 'Solver': [{'Status': 'ok', 'Termination condition': 'optimal', 'Statistics': {'Branch and bound': {'Number of bounded subproblems': 0, 'Number of created subproblems': 0}}, 'Error rc': 0, 'Time': 0.047867536544799805}], 'Solution': [OrderedDict([('number of solutions', 0), ('number of solutions displayed', 0)])]}

In [16]:
print(f"z= {pe.value(M.z)}")
for k in M.x.keys(): 
    print (f"x_{k} = {pe.value(M.x[k])}")


z= 2845522.3880597027
x_verde = 770149.253731343
x_azul = 600000.0
x_comum = 12322388.0597015
